In [ ]:


from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import json
from PIL import Image
from tqdm import tqdm

def process_images_and_labels(input_dir, cropped_output_dir, labels_output_dir):
    os.makedirs(cropped_output_dir, exist_ok=True)
    os.makedirs(labels_output_dir, exist_ok=True)

    for item in tqdm(os.listdir(input_dir), desc='Preparing data'):
        sequence_path = os.path.join(input_dir, item)

        if os.path.isdir(sequence_path) and item.startswith("sequence."):
            image_path = None
            json_path = None

            for file in os.listdir(sequence_path):
                file_path = os.path.join(sequence_path, file)
                if file.endswith(".png") and not image_path:
                    image_path = file_path
                elif file.endswith(".json") and not json_path:
                    json_path = file_path

            if image_path and json_path:

                with open(json_path, 'r') as f:
                    label_data = json.load(f)


                annotations = label_data.get("captures", [])[0].get("annotations", [])


                bbox_annotation = next(
                    (a for a in annotations if a.get("@type") == "type.unity.com/unity.solo.BoundingBox2DAnnotation"),
                    None
                )

                if bbox_annotation:
                    bbox_values = bbox_annotation.get("values", [{}])[0]
                    origin = bbox_values.get("origin")
                    dimensions = bbox_values.get("dimension")

                    if origin and dimensions:
                        x, y = origin
                        w, h = dimensions


                        image = Image.open(image_path)
                        cropped_image = image.crop((x, y, x + w, y + h))


                        output_image_path = os.path.join(cropped_output_dir, f"{item}.png")
                        cropped_image.save(output_image_path)


                        rotation_annotation = next(
                            (a for a in annotations if a.get("@type") == "type.unity.com/unity.solo.BoundingBox3DAnnotation"),
                            None
                        )
                        if rotation_annotation:
                            rotation_values = rotation_annotation.get("values", [{}])[0]
                            quaternion_angles = rotation_values.get("rotation")

                            if quaternion_angles and len(quaternion_angles) == 4:

                                output_label_path = os.path.join(labels_output_dir, f"{item}.txt")
                                with open(output_label_path, 'w') as f:
                                    f.write(" ".join(map(str, quaternion_angles)) + "\n")
                            else:
                                print(f"Missing quaternion angles in {json_path}")
                        else:
                            print(f"No valid 3D bounding box in {json_path}")
                    else:
                        print(f"Missing origin or dimension in {json_path}")
                else:
                    print(f"No valid 2D bounding box in {json_path}")
            else:
                print(f"Skipping folder {sequence_path}: missing required PNG or JSON file.")


input_directory = "/content/drive/MyDrive/CV_FinalProj_Data/shuttle_unzipped/shuttle/shuttle_main"
cropped_images_output_directory = "/content/drive/MyDrive/CV_FinalProj_Data/Shuttle_main_cropped/images/"
quaternion_labels_output_directory = "/content/drive/MyDrive/CV_FinalProj_Data/Shuttle_main_cropped/quaternion_labels/"
process_images_and_labels(input_directory, cropped_images_output_directory, quaternion_labels_output_directory)


Preparing data: 100%|██████████| 20008/20008 [3:08:14<00:00,  1.77it/s]
